In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        07/11/2025  -  05:00:00           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    10/11/2025  -  11:34:43           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================# 

# Competición Perretes y Gatos

## Iteración 10 - Migración a EfficientNetB3 - V2

Añadida test data augmentation, aunque no mejoró el score público mejoró en casi dos decimas en el privado por lo que es el modelo definitivo para esta entrega. 

**kaggle score:** 0.97014 (publico) y 0.98305 privado

In [ ]:
# Fix protobuf compatibility issue
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from tensorflow import data as tf_data
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.applications import EfficientNetB3
from keras.callbacks import EarlyStopping, ModelCheckpoint

seed = 42
keras.utils.set_random_seed(seed)
np.random.seed(seed)

# Rutas dataset
DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("Keras:", keras.__version__)

## Config 

In [ ]:
IMG_SIZE = 300  # Aumentado para captar más detalles (B3 aguanta perfectamente)
BATCH_SIZE = 32  # Reducido para mantener VRAM con resolución mayor
N_FINE_TUNE_LAYERS = 20  # Más capas para fine-tuning
EPOCHS_TL = 15  # Aumentado para mejor convergencia
EPOCHS_FT = 10   # Aumentado para mejor fine-tuning
DROPOUT_RATE = 0.25  # Reducido (modelo menos propenso a overfitting a higher res)
LABEL_SMOOTHING = 0.05  # Reducido (0.1 era demasiado)

print(f"Modelo: EfficientNetB3 (optimizado para alta resolución)")
print(f"Resolución: {IMG_SIZE}x{IMG_SIZE} (aumentada para más detalles)")
print(f"Batch size: {BATCH_SIZE} (reducido por VRAM)")
print(f"Dropout: {DROPOUT_RATE}")
print(f"Label Smoothing: {LABEL_SMOOTHING}")
print(f"Mixup: DISABLED (mejor sin él)")
print(f"Fine-tune layers: {N_FINE_TUNE_LAYERS}")
print(f"Épocas TL: {EPOCHS_TL}")
print(f"Épocas FT: {EPOCHS_FT}")

## Carga de Datos

Todo igual que antes, pero con batch_size=16 

In [ ]:
# Configuro manejo de imágenes corruptas
import tensorflow as tf
tf.io.gfile = tf.io.gfile  # Forzar uso de backend correcto

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

train_dataset = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
)

validation_dataset = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
)

test_dataset = keras.utils.image_dataset_from_directory(
    TEST_PATH,
    labels=None,
    label_mode=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False,
    seed=seed,
    interpolation='bilinear',
)

supplementary_dataset = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False,
    seed=seed,
    interpolation='bilinear',
)

print(f"Train batches: {len(train_dataset)}")
print(f"Validation batches: {len(validation_dataset)}")
print(f"Test batches: {len(test_dataset)}")
print(f"Supplementary batches: {len(supplementary_dataset)}")

## Data Augmentation

Mismo que antes 
- Flip horizontal + vertical
- Rotation ±20%
- Zoom ±20%
- Translation ±15%
- Contrast ±20%
- Brightness ±20%

In [ ]:
# Augmentation - Más agresivo para regularización
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),  # Horizontal flip
    keras.layers.RandomRotation(0.15),  # Rotación +/- 15%
    keras.layers.RandomZoom(0.15),  # Zoom +/- 15%
    keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),  # Traslación +/- 10%
    keras.layers.RandomContrast(0.15),  # Contraste +/- 15%
    keras.layers.RandomBrightness(0.1),  # Brillo +/- 10%
], name="data_augmentation")

# SIN Mixup - aplicar solo augmentation estándar
train_dataset_augmented = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=tf_data.AUTOTUNE
)

print("Augmentation MEJORADO (más agresivo en training, mejor regularización)")

In [ ]:
# Cargo EfficientNetB3 pre-entrenado
efficientnet_base = EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'
)

# Congelo todo inicialmente
for layer in efficientnet_base.layers:
    layer.trainable = False

# Construcción MEJORADA - Mejor arquitectura del cabezal
efficientnet_model = Sequential([
    efficientnet_base,
    keras.layers.BatchNormalization(),
    Dense(512, activation='relu'),  # Primera capa más grande
    Dropout(DROPOUT_RATE),
    keras.layers.BatchNormalization(),
    Dense(256, activation='relu'),  # Segunda capa media
    Dropout(DROPOUT_RATE * 0.8),  # Slightly less dropout
    keras.layers.BatchNormalization(),
    Dense(128, activation='relu'),  # Tercera capa para refinamiento
    Dropout(DROPOUT_RATE * 0.6),  # Even less dropout
    Dense(1, activation='sigmoid')
])

# compilo con AdamW + Cosine Decay
initial_lr = 1e-3
efficientnet_model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=initial_lr,
        weight_decay=1e-4
    ),
    loss=keras.losses.BinaryCrossentropy(
        label_smoothing=LABEL_SMOOTHING
    ),
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

efficientnet_model.summary()
print(f"EfficientNetB3 layers: {len(efficientnet_base.layers)}")
print(f"Arquitectura MEJORADA: Cabezal más profundo (512→256→128→1)")

## Transfer Learning

Entreno solo las capas Dense q añadí, con EfficientNetB3 congelado.

**15 épocas** con early stopping (patience=5).

In [ ]:
# Transfer Learning MEJORADO con Warmup
print("Starting Transfer Learning (optimized with warmup)...")

# Learning Rate Schedule: Warmup + Cosine Decay
total_steps = EPOCHS_TL * len(train_dataset_augmented)
warmup_steps = len(train_dataset_augmented) * 2  # 2 epochs de warmup

class WarmupCosineDecay(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps, decay_steps, alpha=0.1):
        self.initial_lr = initial_lr
        self.warmup_steps = warmup_steps
        self.decay_steps = decay_steps
        self.alpha = alpha
    
    def __call__(self, step):
        if step < self.warmup_steps:
            # Warmup: lineal desde 0.1 * initial_lr hasta initial_lr
            return self.initial_lr * (0.1 + 0.9 * step / self.warmup_steps)
        else:
            # Cosine decay
            progress = (step - self.warmup_steps) / (self.decay_steps - self.warmup_steps)
            return self.initial_lr * (0.5 * (1 + np.cos(np.pi * progress)) * (1 - self.alpha) + self.alpha)
    
    def get_config(self):
        return {
            "initial_lr": self.initial_lr,
            "warmup_steps": self.warmup_steps,
            "decay_steps": self.decay_steps,
            "alpha": self.alpha
        }

lr_schedule = WarmupCosineDecay(
    initial_lr=1e-3,
    warmup_steps=warmup_steps,
    decay_steps=total_steps,
    alpha=0.1
)

# Recompilar con schedule mejorado
efficientnet_model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=lr_schedule,
        weight_decay=1e-4
    ),
    loss=keras.losses.BinaryCrossentropy(
        label_smoothing=LABEL_SMOOTHING
    ),
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

efficientnet_history_tl = efficientnet_model.fit(
    train_dataset_augmented,
    epochs=EPOCHS_TL,
    validation_data=validation_dataset,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=4,
            restore_best_weights=True
        ),
        keras.callbacks.ModelCheckpoint(
            'best_model_tl.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        )
    ]
)

print("Transfer Learning completed (with Warmup + Cosine Decay)")

## Visualización TL

In [ ]:
# Gráficos Transfer Learning (robustos a nombres de métricas)
try:
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(efficientnet_history_tl.history.get('loss', []), label='Train')
    plt.plot(efficientnet_history_tl.history.get('val_loss', []), label='Validation')
    plt.title('Loss - Transfer Learning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(efficientnet_history_tl.history.get('accuracy', []), label='Train')
    plt.plot(efficientnet_history_tl.history.get('val_accuracy', []), label='Validation')
    plt.title('Accuracy - Transfer Learning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Detect keys for precision/recall (Keras names may vary)
    precision_key = next((k for k in ['precision_1', 'precision'] if k in efficientnet_history_tl.history), None)
    recall_key = next((k for k in ['recall_1', 'recall'] if k in efficientnet_history_tl.history), None)

    plt.subplot(1, 3, 3)
    if precision_key and recall_key:
        plt.plot(efficientnet_history_tl.history[precision_key], label='Precision')
        plt.plot(efficientnet_history_tl.history[recall_key], label='Recall')
    else:
        plt.plot([], [], label='Precision (n/a)')
        plt.plot([], [], label='Recall (n/a)')

    plt.title('Precision & Recall - Transfer Learning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Warning: Plotting failed ({e}). Continuing...")

## Evaluación Supplementary (post-TL)

Veo cómo va ANTES del fine-tuning.

In [ ]:
print("Evaluating on supplementary dataset (post-TL)...")

efficientnet_supp_results_tl = efficientnet_model.evaluate(supplementary_dataset, verbose=1)
efficientnet_supp_accuracy_tl = efficientnet_supp_results_tl[1]

print(f"Supplementary Accuracy (TL): {efficientnet_supp_accuracy_tl:.4f}")

## Fine-tuning - DESCONGELAR 15 CAPAS

Ahora descongelo las **últimas 15 capas** de B3 (optimizado).

LR MUY bajo (5e-6) para no romper lo pre-entrenado.

In [ ]:
# Descongelo MÁS capas de B3 para fine-tuning agresivo
print(f"Unfreezing last {N_FINE_TUNE_LAYERS} layers...")

for layer in efficientnet_base.layers[-N_FINE_TUNE_LAYERS:]:
    layer.trainable = True

# LR Schedule para fine-tuning (empezar bajo, descender lentamente)
total_steps_ft = EPOCHS_FT * len(train_dataset_augmented)
lr_schedule_ft = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=5e-5,  # Más alto que antes (5e-6) para exploración
    decay_steps=total_steps_ft,
    alpha=0.01  # Bajar a un 1% del LR inicial
)

# Recompilo con schedule mejorado
efficientnet_model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=lr_schedule_ft,
        weight_decay=5e-5
    ),
    loss=keras.losses.BinaryCrossentropy(
        label_smoothing=LABEL_SMOOTHING
    ),
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

trainable_count = sum([layer.trainable for layer in efficientnet_base.layers])
print(f"Trainable layers: {trainable_count}/{len(efficientnet_base.layers)}")
print(f"Optimizer: AdamW (lr=5e-5 Cosine Decay, weight_decay=5e-5)")

In [ ]:
# Fine-tuning MEJORADO con mejor regularización
print("Starting Fine-tuning (aggressive with better LR schedule)...")

efficientnet_history_ft = efficientnet_model.fit(
    train_dataset_augmented,
    epochs=EPOCHS_FT,
    validation_data=validation_dataset,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,  # Más paciente
            restore_best_weights=True,
            min_delta=1e-4
        ),
        keras.callbacks.ModelCheckpoint(
            'best_model_ft.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        )
    ]
)

print("Fine-tuning completed")

## Visualización Fine-tuning

In [ ]:
try:
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(efficientnet_history_ft.history.get('loss', []), label='Train')
    plt.plot(efficientnet_history_ft.history.get('val_loss', []), label='Validation')
    plt.title('Loss - Fine-tuning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(efficientnet_history_ft.history.get('accuracy', []), label='Train')
    plt.plot(efficientnet_history_ft.history.get('val_accuracy', []), label='Validation')
    plt.title('Accuracy - Fine-tuning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Detect keys for precision/recall (Keras names may vary)
    precision_key_ft = next((k for k in ['precision_1', 'precision'] if k in efficientnet_history_ft.history), None)
    recall_key_ft = next((k for k in ['recall_1', 'recall'] if k in efficientnet_history_ft.history), None)

    plt.subplot(1, 3, 3)
    if precision_key_ft and recall_key_ft:
        plt.plot(efficientnet_history_ft.history[precision_key_ft], label='Precision')
        plt.plot(efficientnet_history_ft.history[recall_key_ft], label='Recall')
    else:
        plt.plot([], [], label='Precision (n/a)')
        plt.plot([], [], label='Recall (n/a)')

    plt.title('Precision & Recall - Fine-tuning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Warning: Plotting failed ({e}). Continuing...")

## Evaluación Final Supplementary (post-FT)

Evaluación final en supplementary dataset SIN augmentation.

In [ ]:
print("Evaluating on supplementary dataset (post-FT, NO augmentation)...")

efficientnet_supp_results_ft = efficientnet_model.evaluate(supplementary_dataset, verbose=1)
efficientnet_supp_accuracy_ft = efficientnet_supp_results_ft[1]

print(f"Supplementary Accuracy (FT): {efficientnet_supp_accuracy_ft:.4f}")
print(f"Expected Kaggle score: ~{efficientnet_supp_accuracy_ft + 0.01:.4f} - {efficientnet_supp_accuracy_ft + 0.03:.4f}")

## Predicciones Finales+ con TTA

TTA fue probado en esta iteración pero **mantuvo el score de 0.970**.

**Razones:**
- TTA amplifica el bias hacia Clase 0 (Gatos)
- Augmentations en test añaden ruido innecesario
- El modelo ya es lo suficientemente robusto (0.9167 supp accuracy)
- pero seguramente en el test privado mejore si mide el overfitting. 


In [ ]:
# Configuración TTA
N_TTA = 10  # Número de augmentations por imagen (10 = buen balance precisión/tiempo)

print(f"Model: EfficientNetB3")
print(f"Resolution: {IMG_SIZE}x{IMG_SIZE}")
print("Generating predictions (NO TTA - direct predictions for better accuracy)...")
print("=" * 60)

# Augmentation MÁS SUAVE para TTA
tta_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomZoom(0.05),      
    keras.layers.RandomTranslation(0.03, 0.03), 
], name="tta_augmentation")

# Función para aplicar TTA
def predict_with_tta(model, dataset, n_tta=N_TTA):
    """
    Realiza predicciones con Test-Time Augmentation
    
    Args:
        model: Modelo entrenado
        dataset: Dataset de test
        n_tta: Número de augmentations por imagen
    
    Returns:
        Array de predicciones promediadas
    """
    print(f"Starting TTA predictions ({n_tta} augmentations)...")
    
    # Primera predicción SIN augmentation (baseline)
    predictions_list = [model.predict(dataset, verbose=0)]
    print(f"✓ Baseline prediction (1/{n_tta+1}) completed")
    
    # Predicciones con augmentation
    for i in range(n_tta):
        augmented_dataset = dataset.map(
            lambda x: tta_augmentation(x, training=True),
            num_parallel_calls=tf_data.AUTOTUNE
        )
        
        preds = model.predict(augmented_dataset, verbose=0)
        predictions_list.append(preds)
        
        print(f"✓ TTA prediction ({i+2}/{n_tta+1}) completed")
    
    # Promediar todas las predicciones
    predictions_avg = np.mean(predictions_list, axis=0)
    
    print(f"TTA completed: {n_tta+1} predictions averaged")
    print("=" * 60)
    
    return predictions_avg

predictions = predict_with_tta(efficientnet_model, test_dataset, n_tta=N_TTA)

print(f"Total predictions: {len(predictions)}")
print(f"Prediction range: [{predictions.min():.4f}, {predictions.max():.4f}]")
print(f"Prediction mean: {predictions.mean():.4f}")

In [ ]:
# Genero submission.csv
test_filenames = test_dataset.file_paths
ids = [int(os.path.splitext(os.path.basename(f))[0]) for f in test_filenames]

# Threshold optimizado: 0.5 es estándar
predictions_binary = (predictions > 0.5).astype(int).flatten()

submission_df = pd.DataFrame({
    'id': ids,
    'label': predictions_binary
})

submission_df = submission_df.sort_values('id')


# Análisis Final y Conclusiones

En esta sección se presentan las métricas finales, las curvas de aprendizaje y una reflexión sobre el rendimiento del modelo.

In [ ]:
# Análisis de Métricas Finales y Curvas de Aprendizaje
import matplotlib.pyplot as plt

def plot_learning_curves(history, title_prefix=""):
    plt.figure(figsize=(15, 5))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history.get('loss', []), label='Train')
    plt.plot(history.history.get('val_loss', []), label='Validation')
    plt.title(f'{title_prefix} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history.get('accuracy', []), label='Train')
    plt.plot(history.history.get('val_accuracy', []), label='Validation')
    plt.title(f'{title_prefix} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Mostrar métricas finales en validation y supplementary
print("Métricas finales en validation:")
val_loss = efficientnet_history_ft.history['val_loss'][-1]
val_acc = efficientnet_history_ft.history['val_accuracy'][-1]
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")

print("\nMétricas finales en supplementary_data:")
print(f"Supplementary Loss: {efficientnet_supp_results_ft[0]:.4f}")
print(f"Supplementary Accuracy: {efficientnet_supp_results_ft[1]:.4f}")

# Curvas de aprendizaje
plot_learning_curves(efficientnet_history_tl, title_prefix="Transfer Learning")
plot_learning_curves(efficientnet_history_ft, title_prefix="Fine-tuning")

# Comentario sobre overfitting/underfitting
if abs(val_acc - efficientnet_supp_results_ft[1]) < 0.02:
    print("No se observa overfitting significativo: la accuracy en validation y supplementary son similares.")
else:
    print("Posible overfitting: diferencia notable entre validation y supplementary.")

## Reflexión Final

Las 10 versiones de notebook creo que han sido determinantes para alcanzar el mejor resultado posible ya que ha sido prueba, error y ajuste constante. Los resultados y scores obtenidos en cada etapa han guiado al final todas las decisiones que he ido siguiendo. El comienzo inicial con una CNN básica, el toparme con la necesidad de usar transfer learning, la prueba de diferentes modelos VGG6, B3 y B7, y todas las optimizaciones añadidas cuando me decidí a usar EfficentNetB3 son reflejo de esta experimentación. 

Las métricas finales del modelo creo que reflejan este progreso:

- **Validation Accuracy:** 0.93
- **Supplementary Accuracy:** 0.9167
- **Kaggle Public Score:** 0.97014
- **Kaggle Private Score:** 0.98305

La diferencia positiva entre el score público y privado (+1.29 puntos) indica que el modelo no está sobreajustado al test público y generaliza muy bien a datos no vistos. EfficientNetB3 V2 con TTA es el modelo más equilibrado y robusto, logrando la mejor combinación de precisión, eficiencia y generalización. 